# 第1ステージ　微分を自動で求める

## ステップ1　箱としての変数

In [ ]:
import numpy as np

# 機械学習のシステムはデ、基礎とするータ構造に「多次元配列」を使用する。
# VariableクラスはNumpyの多次元配列のみを扱える仕様とする
class Variable:
    def __init__(self, data):
        self.data = data


data = np.array(1.0)
x = Variable(data)
print(x.data)

x.data = np.array(2.0)
print(x.data)

## ステップ2　変数を生み出す関数

In [3]:
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data


# Functionクラスは基底クラスとして、全ての関数に共通する機能を実装
# 具体的な機能（関数）はFunctionクラスを継承したクラスで実装
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output

    def forward(self, in_data):
        # Functionクラスからの直接forwardメソッドを使用はNG
        raise NotImplementedError()


# Functionクラスを継承
class Square(Function):
    def forward(self, x):
        return x ** 2


x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


## ステップ3　関数の連結

In [2]:
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        return output

    def forward(self, x):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2


class Exp(Function):
    def forward(self, x):
        return np.exp(x)


# 関数と変数が交互に並ぶ計算グラフで表すことができる
# 複数の関数によって構成される関数を合成関数と呼ぶ
# 計算グラフは効率よく各変数の微分を求めることができる
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


## ステップ4　数値微分

In [4]:
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        return x ** 2


class Exp(Function):
    def forward(self, x):
        return np.exp(x)


# 数値微分
# f:微分対象の関数()Functionインスタンス
# x:微分を求めるポイント
# eps:Δx
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)


f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)


# 合成関数の微分
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))


x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

4.000000000004
3.2974426293330694


## ステップ6　手作業によるバックプロパゲーション

In [1]:
# 数値微分では精度と計算コストの点で問題あり
# バックプロパゲーション法では本問題を解決
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None  # 微分値


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input  # 入力値保持
        return output

    # 順伝播
    def forward(self, x):
        raise NotImplementedError()

    # 逆伝播
    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx


A = Square()
B = Exp()
C = Square()

# 順伝播
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 逆伝播
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)

print(x.grad)

3.297442541400256
